In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import dash
import pandas as pd

# Import the AnimalShelter class from your CRUD module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connection Variables
username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'      
port = 32450
DB = 'AAC'
COL = 'animals'

# Instantiate AnimalShelter
shelter = AnimalShelter(user=username, password=password, host=host, port=port)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# # MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
    
## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = '/home/rikkixaysanas_snhu/Desktop/AnimalShelter/Grazioso Salvare Logo.png'  # Image FilePath

# Encode the image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

app.layout = html.Div([
    # Hidden div for potential future use
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Displays Grazioso Logo image with a URL anchor
    html.A(
        href="https://www.snhu.edu",  # SNHU webpage
        children=html.Img(
            src='data:image/png;base64,{}'.format(encoded_image), 
            style={'width': '200px', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}
        )
    ),

    # My Unique identifier with navy blue color and cursive font
    html.Div(
        "Created by: Rikki Xaysanasy",
        style={
            'textAlign': 'center',
            'padding': '10px',
            'fontSize': '18px',
            'fontFamily': 'cursive',
            'color': 'navy'
        }
    ),
    
    # Main Title
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),

    html.Hr(),
    
    # Buttons for filtering data four options
    html.Div(className='buttonRow',
             style={'display': 'flex', 'justify-content': 'space-around'},
             children=[
                 html.Button(id='submit-button-all', n_clicks=0, children='Reset'),
                 html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                 html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
                 html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
             ]),
    
    # DataTable to display the data
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode='multi',
        row_selectable='multi',
        row_deletable=True,
        selected_rows=[],
        page_action='native',
        page_current= 0,
        page_size= 10,
    ),
    
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    
])

# Callback for handling button clicks to filter the DataTable
@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-all', 'n_clicks'),
     Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks')]
)
def update_dashboard(reset_clicks, water_rescue_clicks, mountain_rescue_clicks, disaster_rescue_clicks):
    # This variable helps us figure out which button was clicked
    button = dash.callback_context

    # If no button was clicked ; show all data
    if not button.triggered:
        # Get all records from the database
        df = pd.DataFrame.from_records(shelter.read({}))
    else:
        # Get the ID of the button that was clicked
        button_id = button.triggered[0]['prop_id'].split('.')[0]

        # If Water Rescue button was clicked
        if button_id == 'submit-button-one':
            # Query to find dogs suitable for water rescue
            query = {
                "$and": [
                    # Look for breeds all with *Retrievers or *Newfoundland
                    {"breed": {"$regex": ".*Retriever.*|.*Newfoundland.*", "$options": "i"}},
                    # Only intact females
                    {"sex_upon_outcome": "Intact Female"},
                    # Age range between 26 and 156 weeks
                    {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
                ]
            }
        # If Mountain or Wilderness Rescue button was clicked
        elif button_id == 'submit-button-two':
            # Query for mountain or wilderness rescue dogs
            query = {
                "$and": [
                    # Look for all breeds with *Shepherds, *Malamutes, etc etc
                    {"breed": {"$regex": ".*Shepherd.*|.*Malamute.*|.*Sheepdog.*|.*Husky.*|.*Rottweiler.*", "$options": "i"}},
                    # Only intact males
                    {"sex_upon_outcome": "Intact Male"},
                    # Age range between 26 and 156 weeks
                    {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
                ]
            }
        # If Disaster Rescue or Individual Tracking button was clicked
        elif button_id == 'submit-button-three':
            # Query for disaster rescue or tracking dogs
            query = {
                "$and": [
                    # Look for all breeds with  *Pinschers, *Shepherds, etc etc
                    {"breed": {"$regex": ".*Pinscher.*|.*Shepherd.*|.*Retriever.*|.*Bloodhound.*|.*Rottweiler.*", "$options": "i"}},
                    # Only look for intact males
                    {"sex_upon_outcome": "Intact Male"},
                    # Age range between 20 and 300 weeks
                    {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
                ]
            }
        else:
            # If the 'Reset' button was clicked, returns to unfiltered list
            query = {}

        # Get the filtered records based on the query
        df = pd.DataFrame.from_records(shelter.read(query))

    # Clean up the data by removing the '_id' column if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Return the filtered data to be displayed in the table
    return df.to_dict('records')

# Callback to update the pie chart based on the filtered data
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_graph(data):
    if data is None or len(data) == 0:
        return [html.Div("No data to display.")]

    dff = pd.DataFrame.from_dict(data)

    # Group by breed and count the number of each breed in the filtered data
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    # Creates the pie chart
    fig = px.pie(breed_counts, values='count', names='breed', title='Breed Visual Pie Chart')

    return dcc.Graph(figure=fig)

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or index is None or len(viewData) == 0:
        return

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0] if index else 0

    # Ensure columns 13 (latitude) and 14 (longitude) exist
    if len(dff.columns) < 15:
        return [html.Div("Geolocation data unavailable for the selected entry.")]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:20065/
